In [66]:
import pandas as pd
import numpy as np

def process_23andme_data(position_info_path, andme_data_path, output_path):
    def read_data(position_info_path, andme_data_path):
        position_info = pd.read_csv(position_info_path)
        andme_data = pd.read_csv(andme_data_path, sep='\t', comment='#', header=None, names=['rsid', 'chromosome', 'position', 'genotype'])
        return position_info, andme_data

    def merge_data(position_info, andme_data):
        return pd.merge(position_info, andme_data, left_on='RS_Number', right_on='rsid', how='left')

    def create_allele_columns(data):
        data['maternal_allele'] = data['genotype'].str[0]
        data['paternal_allele'] = data['genotype'].str[1]
        return data

    def calc_dosage(allele, alt_allele):
        return np.where(pd.isna(allele) | pd.isna(alt_allele), np.nan, np.where(allele == alt_allele, 1, 0))

    def apply_dosage_calculation(data):
        data['maternal_dosage'] = calc_dosage(data['maternal_allele'], data['Alt'])
        data['paternal_dosage'] = calc_dosage(data['paternal_allele'], data['Alt'])
        data['unphased_dosage'] = data['maternal_dosage'] + data['paternal_dosage']
        return data

    def create_phased_columns(data):
        coord_prefix = data['Coord'].str.split(':').str[0]
        position = data['Coord'].str.split(':').str[1]
        ref = data['Alleles'].apply(lambda x: x.split('/')[0][1])
        alt = data['Alleles'].apply(lambda x: x.split('/')[1][0])
        data['phased_column_maternal'] = coord_prefix + '_' + position + '_' + ref + '_' + alt + '_maternal'
        data['phased_column_paternal'] = coord_prefix + '_' + position + '_' + ref + '_' + alt + '_paternal'
        data['unphased_column'] = coord_prefix + '_' + position + '_' + ref + '_' + alt + '_combined'
        return data

    def create_output_dataframe(data):
        new_data = {
            **dict(zip(data["phased_column_maternal"], data["maternal_dosage"])),
            **dict(zip(data["phased_column_paternal"], data["paternal_dosage"])),
            **dict(zip(data["unphased_column"], data["unphased_dosage"]))
        }
        df = pd.DataFrame([new_data])
        return df.sort_index(axis=1)

    def save_to_csv(df, output_path):
        df.to_csv(output_path, index=False)

    # Pipeline execution
    position_info, andme_data = read_data(position_info_path, andme_data_path)
    merged_data = merge_data(position_info, andme_data)
    merged_data = create_allele_columns(merged_data)
    merged_data = apply_dosage_calculation(merged_data)
    merged_data = create_phased_columns(merged_data)
    output_df = create_output_dataframe(merged_data)
    save_to_csv(output_df, output_path)

# Example usage:
process_23andme_data('../../Data/Filtered_unphased_training_data_union_final/23andMe_matching_variants_updated.csv', 
                     '../../Data/23andMe_files/6800.23andme.10042.txt', 
                     'output.csv')


/var/folders/yg/982c10113957_2gb06y92y35sx509h/T/ipykernel_11873/2964836441.py:7: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  andme_data = pd.read_csv(andme_data_path, sep='\t', comment='#', header=None, names=['rsid', 'chromosome', 'position', 'genotype'])


In [ ]:
# Create the new dictionary with the desired structure


In [ ]:
# Create the new dictionary with the desired structure
